无论我们定义什么样的Node、Edge、Graph，核心目的都是希望通过对State的操作，以达到我们想要的效果。
### 1.关于State的定义
LangGraph中的state，可以用TypedDict、Pydantic Model，或者dataclass来定义，其中前两种比较常用。
#### 1.1 TypedDict：轻量级的选择
TypedDict主打一个轻量级和简单直接。它本质上就是Python标准库typing模块中的一个类型提示工具，不需要安装额外的依赖，使用起来非常方便。
```python
from typing import TypedDict

class AppState(TypedDict):
    question: str
    answer: str
```
注意：typing本质上只是一种类型提示，不会对实际运行时的数据进行类型检查。如果需要在运行时进行类型检查，建议使用Pydantic Model。
#### 1.2 Pydantic Model：更强大的选择
Pydantic是一个强大的数据验证库，它可以在项目运行时对数据进行验证，确保数据的类型和格式符合预期。这对于构建健壮的AI Agent来说非常重要。
```python
from pydantic import BaseModel

class AppState(BaseModel):
    question: str
    answer: str
    key_1: int = Field(ge=0, le=100)
    key_2: str = "默认值"
```
注意：Pydantic Model需要安装pydantic库。可以使用以下命令安装：
```bash
pip install pydantic
```
与TypedDict相比，Pydantic Model提供了几个非常实用的功能：
1. 数据验证：Pydantic Model可以在运行时对数据进行验证，确保数据的类型和格式符合预期。
2. 默认值：可以为Pydantic Model的字段设置默认值，当数据中没有对应字段时，会使用默认值。
3. 字段约束：可以为Pydantic Model的字段设置约束条件，例如字段的取值范围、字段是否必填等。
4. 数据转换：Pydantic Model可以将数据转换为Python对象，方便在项目中使用。

在实际开发过程中：
- TypedDict：项目比较简单，不需要复杂的验证逻辑，或者你希望保持代码的轻量级；
- Pydantic Model：项目需要复杂的数据验证逻辑，或者你的state结构比较复杂，需要字段间的依赖关系。

### 2.关于State的更新
在LangGraph中，State的更新其实是一个两阶段的过程：
1. 第一阶段：节点函数处理-Node函数接收当前的state，根据业务逻辑进行处理，计算出需要更新的新值。
2. 第二阶段：Reducer决定更新方式-Reducer函数决定如何将节点返回的新值更新到state中，是覆盖（override）还是拼接（append）等。
#### 2.1 第一阶段：节点函数处理
节点函数接收当前的state作为输入，根据业务逻辑进行处理，返回一个结果，包含需要更新的state字段和新值。
```python
def check_node(state: AppNode) -> AppNode:
    """检查节点"""
    dice_roll = random.randint(1, 6)
    if dice_roll % 2 == 0:
        return {"answer": "偶数"}
    else:
        return {"answer": "奇数"}
```
但这时，我们还无法判断这个新值将如何对state产生影响，因为这还得由定义state时设置的Reducer来决定。
#### 2.2 Reducer决定更新方式
##### 2.2.1 Reducer的基本用法
常见的一个Reducer,add_messages:
```python
from typing import Annotated, List
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
```
这里的关键是Annotated的使用，Annotated是Python的类型注解工具，它允许我们为类型添加额外的元数据。在LangGraph中，我们用它来指定Reducer函数。
add_messages的作用是：
- 将新消息追加到现有消息列表的末尾；
- 如果新消息的ID与旧消息相同，则更新旧消息而不是追加
除了add_messages，LangGraph中还有其他的一些内置的Reducer，比如：
1. operator.add：用于数值累加，将新值加到旧值上。比如统计总分、计数等场景。
```python
from typing import Annotated, TypedDict
from operator import add

class ScoreState(TypedDict):
    total_score: Annotated[int, add]  # 每次返回的分数会累加到总分上
```
2. operator.extend：用于列表拼接，将新列表追加到旧列表的末尾。比如收集所有生成的文本、拼接所有生成的消息等场景。
```python
from typing import Annotated, TypedDict, List
from operator import extend

class TextState(TypedDict):
    generated_texts: Annotated[List[str], extend]  # 每次返回的文本会追加到已有的文本列表中
```
##### 2.2.2 自定义Reducer
当内置的Reducer无法满足需求时，我们可以自定义Reducer函数实现对state更新过程的控制。自定义Reducer的过程非常简单，只需要定义一个函数即可。
参数：Reducer必须接收两个参数，依次从左到右为：
- old_value: state中该字段的当前值
- new_value: 节点函数返回的新值
返回值：返回合并后的结果，类型应该与字段类型一致。
```python
from typing import Annotated, TypedDict

def take_max(old_value: int, new_value: int) -> int:
    """自定义Reducer：取新旧值中的较大值"""
    return max(old_value, new_value)

class AppState(TypedDict):
    max_score: Annotated[int, take_max]  # 每次返回的分数会取较大值
```
##### 2.2.3 Overwite：强制覆盖机制
在某些场景下，我们可能希望忽略Reducer函数，直接使用新值覆盖state中的旧值。此时可以使用Overwrite机制。
Overwrite允许我们在节点返回时，强制忽略已设定的Reducer，直接覆盖state中的值。
LangGraph提供了两种使用overwrite的方式：
1. 使用Overwrite类型包装值：
```python
from langgraph.graph import Overwrite

def reset_node(state: ChatState) -> ChatState:
    """重置消息列表，忽略add_messages的Reducer"""
    return {"messages": Overwrite([])}  # 直接覆盖为空列表，忽略add_messages的拼接逻辑
```
2. 使用 __overwrite__ 键：
```python
def reset_node(state: ChatState) -> ChatState:
    """重置消息列表，忽略add_messages的Reducer"""
    return {"messages": {"__overwrite__": []}}  # 效果与上面相同
```
overwrite在以下场景特别有用：
- 重置状态：需要清空累积的数据（比如重置历史消息）
- 强制替换：需要完全替换某个字段的值，不考虑之前的累积结果
- 特殊处理：某些节点需要绕过Reducer的常规逻辑
注意，如果不知道Reducer，LangGraph默认就会直接覆盖（相当于自动使用overwrite），所以overwrite主要用于在已设定Reducer的情况下强制覆盖的场景。
### 3.State的管理
在某些场景下，我们可能希望对外暴露的接口与内部使用的State结构有所不同。
这个时候，我们就可以使用input schema和output schema来定义Graph的输入和输出格式。
#### 3.1 基本用法
假设有一个内部的State，结构比较复杂：
```python
from typing import TypedDict, List

class InternalState(TypedDict):
    user_name: str
    user_age: int
    messages: List[str]
    result: str
    status: str
    internal_counter: int
    debug_info: dict
```
我们希望用户调用Graph时，只需要传入用户名和年龄，而不需要关心内部的messages、internal_counter等字段。
这时，我们可以使用input schema来定义Graph的输入格式：
```python
from typing import TypedDict

class InputSchema(TypedDict):
    """用户输入格式"""
    user_name: str
    user_age: int

class OutputSchema(TypedDict):
    """用户输出格式"""
    result: str
    status: str
```
然后在创建Graph时指定这些schema：
```python
from langgraph.graph import StateGraph

graph = StateGraph(InternalState, input_schema=InputSchema, output_schema=OutputSchema)
```
#### 3.2 效果说明
使用input/output schema之后，Graph的行为会发生以下变化：
##### 3.2.1 输入转换
用户输入时，只需要传入InputSchema格式的数据
```python
input_data = InputSchema(user_name="张三", user_age=30)
result = app.invoke(input_data)
```
LangGraph会自动将InputSchema转换为InternalState，未输入的地方会自动初始化为空值。转换后的内部state为：
```python
InternalState(
    user_name="张三",
    user_age=30,
    messages=[],
    result="",
    status="",
    internal_counter=0,
    debug_info={}
)
```
##### 3.2.2 输出转换
当节点处理完成后，LangGraph会将InternalState转换为OutputSchema格式的输出。
例如，假设节点处理完成后，InternalState为：
```python
InternalState(
    user_name="张三",
    user_age=30,
    messages=["你好", "你好"],
    result="你好",
    status="success",
    internal_counter=2,
    debug_info={"step": 2}
)
```
OutputSchema像一个筛选器一样，只会向用户返回在其中定义的键值。则转换后的OutputSchema为：
```python
OutputSchema(
    result="你好",
    status="success"
)
```
#### 3.3 private schema
有时候，我们希望在Graph的执行过程中使用一些中间状态，这些状态只在节点之间传递，但不应该出现在Graph的最终输出中。比如：
1. 中间计算结果：某些节点需要计算中间值，这些值会被后续节点使用，但不需要对外暴露；
2. 临时数据存储：在Graph执行过程中需要临时存储一些数据，用于节点间的信息传递；
3. 内部状态管理：需要跟踪Graph内部的执行状态，但这些状态对用户来说是不必要的。
LangGraph提供了private state机制来实现这个需求。
##### 3.3.1 基本用法
private state的使用方法很简单，就是定义一个独立的PrivateState类型，然后再节点函数中使用它，关键点在于：
- 定义独立的PrivateState类型，包含需要在节点间传递但不对外暴露的字段
- 节点函数可以接收PrivateState作为输入，也可以返回PrivateState作为输出
- PrivateState中的字段会在节点间正常传递，但不会出现在Graph的最终输出中
```python
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# 定义Graph的整体状态：包含公有字段和私有字段
classOverallState(TypedDict):
    user_input: str      # 公有字段，会出现在最终输出中
    graph_output: str    # 公有字段，会出现在最终输出中
    foo: str            # 公有字段，会出现在最终输出中

# 定义私有状态：只在节点间传递，不会出现在最终输出中
classPrivateState(TypedDict):
    bar: str            # 私有字段，只在节点间传递

# 节点1：处理用户输入，写入OverallState
defnode_1(state: OverallState) -> OverallState:
    # 处理用户输入，生成中间结果
    return {"foo": state["user_input"] + " name"}

# 节点2：读取OverallState，写入PrivateState
defnode_2(state: OverallState) -> PrivateState:
    # 从OverallState读取foo，计算中间值并存入PrivateState
    return {"bar": state["foo"] + " is"}

# 节点3：读取PrivateState，写入OverallState
defnode_3(state: PrivateState) -> OverallState:
    # 从PrivateState读取bar，生成最终输出
    return {"graph_output": state["bar"] + " Lance"}

# 创建Graph，只使用OverallState作为主状态
builder = StateGraph(OverallState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

graph = builder.compile()

# 调用Graph
result = graph.invoke({"user_input": "My"})
print(result)
```
私有状态在节点间正常传递
私有状态不会出现在最终输出中

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# 定义Graph的整体状态：包含公有字段和私有字段
class OverallState(TypedDict):
    user_input: str  # 公有字段，会出现在最终输出中
    graph_output: str  # 公有字段，会出现在最终输出中
    foo: str  # 公有字段，会出现在最终输出中


# 定义私有状态：只在节点间传递，不会出现在最终输出中
class PrivateState(TypedDict):
    bar: str  # 私有字段，只在节点间传递



# 节点1：处理用户输入，写入OverallState
def node_1(state: OverallState) -> OverallState:
    # 处理用户输入，生成中间结果
    return {"foo": state["user_input"] + " name"}


# 节点2：读取OverallState，写入PrivateState
def node_2(state: OverallState) -> PrivateState:
    # 从OverallState读取foo，计算中间值并存入PrivateState
    return {"bar": state["foo"] + " is"}



# 节点3：读取PrivateState，写入OverallState
def node_3(state: PrivateState) -> OverallState:
    # 从PrivateState读取bar，生成最终输出
    return {"graph_output": state["bar"] + " Lance"}

# 创建Graph，只使用OverallState作为主状态
builder = StateGraph(OverallState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

graph = builder.compile()

# 调用Graph
result = graph.invoke({"user_input": "My"})
print(result)

{'user_input': 'My', 'graph_output': 'My name is Lance', 'foo': 'My name'}
